#### Import required libraries\modules

In [2]:
import numpy as np 
import pandas as pd
import requests
import json
import matplotlib.cm as cm
import matplotlib.colors as colors
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
!pip install beautifulsoup4
from bs4 import BeautifulSoup 
!pip install geopy
from geopy.geocoders import Nominatim
!pip install folium
import folium
import io

/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method

     |████████████████████████████████| 115 kB 4.5 MB/s eta 0:00:01
     |████████████████████████████████| 112 kB 4.6 MB/s eta 0:00:01
     |████████████████████████████████| 94 kB 2.3 MB/s eta 0:00:011


#### Okey, we import all that need. Lets start main work, first -  write a function, to clean our data set, and prepare data to analysis.

In [4]:
def parse_html_table(table):
    n_columns = 0
    n_rows=0
    column_names = []

    # Find number of rows and columns
    # we also find the column titles if we can
    for row in table.find_all('tr'):
        
        # Determine the number of rows in the table
        td_tags = row.find_all('td')
        if len(td_tags) > 0:
            n_rows+=1
            if n_columns == 0:
                # Set the number of columns for our table
                n_columns = len(td_tags)
                
        # Handle column names if we find them
        th_tags = row.find_all('th') 
        if len(th_tags) > 0 and len(column_names) == 0:
            for th in th_tags:
                column_names.append(th.get_text())

    # Safeguard on Column Titles
    if len(column_names) > 0 and len(column_names) != n_columns:
        raise Exception("Column titles do not match the number of columns")

    columns = column_names if len(column_names) > 0 else range(0,n_columns)
    df = pd.DataFrame(columns = columns,
                        index= range(0,n_rows))
    row_marker = 0
    for row in table.find_all('tr'):
        column_marker = 0
        columns = row.find_all('td')
        for column in columns:
            df.iat[row_marker,column_marker] = column.get_text()
            column_marker += 1
        if len(columns) > 0:
            row_marker += 1
            
    # Convert to float if possible
    for col in df:
        try:
            df[col] = df[col].astype(float)
        except ValueError:
            pass
    
    return df

In [5]:
# Load page with Moscow Boroughs
url = "https://gis-lab.info/qa/moscow-atd.html"
response = requests.get(url)
print('Url "{}" requested. '.format(url))
    
# Take second HTML table with districts from the page and parse it into dataframe
print('Try to extract dataset from HTML table'.format(url))
soup = BeautifulSoup(response.text, 'html.parser')
tables = soup.findAll('table', { 'class' : 'wikitable sortable' }, limit=2) 
Moscow_df = parse_html_table(tables[1])  
print('Success extract dataset from HTML table'.format(url))

# Define columns for dataframe
Moscow_df.columns=["Borough_index", "Borough_Name", "District_Name", "Borough_Type", "OKATO_Borough_Code", "OKTMO_District_Code"]
Moscow_df

Url "https://gis-lab.info/qa/moscow-atd.html" requested. 
Try to extract dataset from HTML table
Success extract dataset from HTML table


,Borough_index,Borough_Name,District_Name,Borough_Type,OKATO_Borough_Code,OKTMO_District_Code
0,1.0,Академический\n,ЮЗАО\n,Муниципальный округ\n,45293554.0,45397000.0
1,2.0,Алексеевский\n,СВАО\n,Муниципальный округ\n,45280552.0,45349000.0
2,3.0,Алтуфьевский\n,СВАО\n,Муниципальный округ\n,45280554.0,45350000.0
3,4.0,Арбат\n,ЦАО\n,Муниципальный округ\n,45286552.0,45374000.0
4,5.0,Аэропорт\n,САО\n,Муниципальный округ\n,45277553.0,45333000.0
...,...,...,...,...,...,...
141,142.0,Роговское\n,Троицкий\n,Поселение\n,45298575.0,45956000.0
142,143.0,Рязановское\n,Новомосковский\n,Поселение\n,45297571.0,45957000.0
143,144.0,Сосенское\n,Новомосковский\n,Поселение\n,45297574.0,45958000.0
144,145.0,Филимонковское\n,Новомосковский\n,Поселение\n,45297577.0,45959000.0


##### Look preety, but need to 'washing' next. Need to delete "\n" symbols, and turn data to right datatype

In [5]:
# Drop Borough_index colums 
# Moscow_df.drop("Borough_index", axis=1, inplace=True)

Moscow_df.replace('\n', '', regex=True, inplace=True)

# convert float to int for code columns
Moscow_df["OKATO_Borough_Code"] = Moscow_df["OKATO_Borough_Code"].astype(int)
Moscow_df["OKTMO_District_Code"] = Moscow_df["OKTMO_District_Code"].astype(int)

Moscow_df

,Borough_index,Borough_Name,District_Name,Borough_Type,OKATO_Borough_Code,OKTMO_District_Code
0,1.0,Академический,ЮЗАО,Муниципальный округ,45293554,45397000
1,2.0,Алексеевский,СВАО,Муниципальный округ,45280552,45349000
2,3.0,Алтуфьевский,СВАО,Муниципальный округ,45280554,45350000
3,4.0,Арбат,ЦАО,Муниципальный округ,45286552,45374000
4,5.0,Аэропорт,САО,Муниципальный округ,45277553,45333000
...,...,...,...,...,...,...
141,142.0,Роговское,Троицкий,Поселение,45298575,45956000
142,143.0,Рязановское,Новомосковский,Поселение,45297571,45957000
143,144.0,Сосенское,Новомосковский,Поселение,45297574,45958000
144,145.0,Филимонковское,Новомосковский,Поселение,45297577,45959000


#### Now need to added geographic coordinates (The Nominatim is not stable to catching data about russing borough)

In [7]:
# instantiate the dataframe
Moscow_coord_df = pd.DataFrame(columns=['Borough_Name', 'Latitude', 'Longitude'] )

# create class instance of Nominatim
geolocator = Nominatim(user_agent="foursquare_agent", timeout=2)

# The Nominatim is not stable, so catch exception and in error load previously saved dataframe
try:
    # loop frough all Boroughs
    for Borough_Name, Borough_Type, District_Name in zip(Moscow_df['Borough_Name'], Moscow_df['Borough_Type'], Moscow_df['District_Name']):
        address = '{}, {}, {},'.format(Borough_Name, Borough_Type, District_Name)
        print(address, end='')

        location = None

        # make up to 10 attempts
        for x in range(0, 1):
            print('.', end='')
            try:
                location = geolocator.geocode(address)
                if location is not None:
                    print(' - coordinate are {}, {}'.format(location.latitude, location.longitude))
                    latitude = location.latitude
                    longitude = location.longitude
                    Moscow_coord_df = Moscow_coord_df.append({'Borough_Name': Borough_Name, 'Latitude': latitude, 'Longitude': longitude}, ignore_index=True) 
                    break
            except Exception as err:
                print('')
                print(type(err))
                print(err) 
                raise

except Exception as err:
    print('')
    print(err)

# Take a look at the dataframe
print('Take a look at the dataframe'.format(url))
print(Moscow_coord_df.head())
print(Moscow_coord_df.shape)

print('Take a look at the dataframe data types'.format(url))
print(Moscow_coord_df.dtypes)

Академический, Муниципальный округ, ЮЗАО,. - coordinate are 55.67864455, 37.57158238499227
Алексеевский, Муниципальный округ, СВАО,. - coordinate are 55.82502585, 37.65543633089784
Алтуфьевский, Муниципальный округ, СВАО,. - coordinate are 55.87421245, 37.57629659675598
Арбат, Муниципальный округ, ЦАО,.Аэропорт, Муниципальный округ, САО,. - coordinate are 55.8004021, 37.5331563
Бабушкинский, Муниципальный округ, СВАО,. - coordinate are 55.88005615, 37.66771877626958
Басманный, Муниципальный округ, ЦАО,. - coordinate are 55.7704524, 37.6688603
Беговой, Муниципальный округ, САО,. - coordinate are 55.7785588, 37.553486
Бескудниковский, Муниципальный округ, САО,. - coordinate are 55.8575791, 37.5602777
Бибирево, Муниципальный округ, СВАО,. - coordinate are 55.90745695, 37.58785394431098
Бирюлёво Восточное, Муниципальный округ, ЮАО,.Бирюлёво Западное, Муниципальный округ, ЮАО,.Богородское, Муниципальный округ, ВАО,. - coordinate are 55.8197386, 37.7159451
Братеево, Муниципальный округ, ЮАО,

#### Checking output data

In [8]:
Moscow_coord_df

,Borough_Name,Latitude,Longitude
0,Академический,55.678645,37.571582
1,Алексеевский,55.825026,37.655436
2,Алтуфьевский,55.874212,37.576297
3,Аэропорт,55.800402,37.533156
4,Бабушкинский,55.880056,37.667719
...,...,...,...
92,Марушкинское,55.586240,37.265358
93,Московский,55.602601,37.347918
94,Первомайское,57.082273,86.186415
95,Сосенское,55.566761,37.466225


#### Dataframe looks good, now find coordinates of Moscow

In [9]:
address = "Moscow"
geolocator = Nominatim(user_agent='moscow_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Moscow are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Moscow are 55.7504461, 37.6174943.


#### Loading data of borderlands of borough

In [10]:
# download geojson file
url = 'http://gis-lab.info/data/mos-adm/mo.geojson'
try:
    print('Try to request url "{}"'.format(url))
    download_file = requests.get(url)
    print('Url "{}" requested. '.format(url))

    mo_geojson_utf8 = 'data\mo.geojson.utf8'
    open(mo_geojson_utf8, 'wb').write(download_file.content)    
    print('GeoJSON file downloaded!')

    mo_geojson = 'data\mo.geojson'

    # Encode file from utf8 to cp1251 as my computer use Russian locale
    f = open(mo_geojson, "wb")
    for line in open(mo_geojson_utf8, "rb"):
        f.write(line.decode('u8').encode('cp1251', 'ignore'))
    f = open(mo_geojson, "wb")
    for line in open(mo_geojson_utf8, "rb"):
        f.write(line.decode('u8').encode('cp1251', 'ignore'))

    # validate geojson file
    with open(mo_geojson) as json_file:
        data = json_file.read()
        try:
            data = json.loads(data)
        except ValueError as e:
            print('invalid json: %s' % e)

except: 
    print('Request Url "{}" failed'.format(url))
    mo_geojson = 'data\mo.geojson'
    print('GeoJSON file downloaded!')

Try to request url "http://gis-lab.info/data/mos-adm/mo.geojson"
Url "http://gis-lab.info/data/mos-adm/mo.geojson" requested. 
GeoJSON file downloaded!
Request Url "http://gis-lab.info/data/mos-adm/mo.geojson" failed
GeoJSON file downloaded!


#### In this case we load and parse data about mean value about flats in different Moscow borough  

In [11]:
# Load page with Moscow Boroughs Housing Price
url = "https://www.mirkvartir.ru/journal/analytics/2018/02/25/reiting-raionov-moskvi-po-stoimosti-kvartir"
try:
    print('Try to request url "{}"'.format(url))
    response = requests.get(url)
    print('Url "{}" requested. '.format(url))

    # Take first HTML table with districts from the page and parse it into dataframe
    soup = BeautifulSoup(response.text, 'html.parser')
    tables = soup.findAll('table', limit=1) 
    Moscow_housing_price_df = parse_html_table(tables[0])  

except Exception as err: 
    print('Request Url "{}" failed. Load previously saved dataframe'.format(url))
    print('Error is: {}'.format(err))

Try to request url "https://www.mirkvartir.ru/journal/analytics/2018/02/25/reiting-raionov-moskvi-po-stoimosti-kvartir"
Url "https://www.mirkvartir.ru/journal/analytics/2018/02/25/reiting-raionov-moskvi-po-stoimosti-kvartir" requested. 


In [12]:
Moscow_housing_price_df

,0,1,2,3,4,5
0,№,Район,"Цена, руб./кв. м",Прирост за год,"Цена квартиры, руб.",Прирост за год
1,1,Арбат,438568,"−0,20%",33702123,"0,10%"
2,2,Хамовники,425741,"4,50%",27196303,"5,20%"
3,3,Якиманка,404471,"1,30%",26920221,"4,10%"
4,4,Замоскворечье,398544,"3,80%",26910141,"5,70%"
...,...,...,...,...,...,...
139,139,Михайлово-Ярцевское,79454,"−6,90%",4208693,"−2,10%"
140,140,Щаповское,75321,"−6,40%",3860212,"−3,30%"
141,141,Новофедоровское,70874,"−4,20%",3573741,"−3,10%"
142,142,Кленовское,68768,"−5,10%",3415926,"−3,80%"


#### Looks good, but need little washing of this dataframe

In [13]:
print('Clear the dataframe')

# Drop some unused colums 
Moscow_housing_price_df.drop([Moscow_housing_price_df.columns[0], Moscow_housing_price_df.columns[3], Moscow_housing_price_df.columns[4], Moscow_housing_price_df.columns[5]], axis=1, inplace=True)
Moscow_housing_price_df.drop(0, axis=0, inplace=True)

# Set columns for dataframe
Moscow_housing_price_df.columns=["Borough_Name", "Borough_Housing_Price"]

# Clear Borough Name from additional information
Moscow_housing_price_df["Borough_Name"] = Moscow_housing_price_df["Borough_Name"].str.strip(' \n\t')

# Replace '\n' in some columns
Moscow_housing_price_df.replace('\n', '', regex=True, inplace=True)

# Convert from string to numeric
Moscow_housing_price_df["Borough_Housing_Price"] = Moscow_housing_price_df["Borough_Housing_Price"].astype(int)

# replace some Borough_Name as of russian letters "е" and "ё" and change places of some words 
Moscow_housing_price_df["Borough_Name"].replace('Бирюлево Восточное', 'Бирюлёво Восточное', regex=True, inplace=True)
Moscow_housing_price_df["Borough_Name"].replace('Бирюлево-Западное', 'Бирюлёво Западное', regex=True, inplace=True)
Moscow_housing_price_df["Borough_Name"].replace('Дегунино Восточное', 'Восточное Дегунино', regex=True, inplace=True)
Moscow_housing_price_df["Borough_Name"].replace('Измайлово Восточное', 'Восточное Измайлово', regex=True, inplace=True)
Moscow_housing_price_df["Borough_Name"].replace('Дегунино Западное', 'Западное Дегунино', regex=True, inplace=True)
Moscow_housing_price_df["Borough_Name"].replace('Савеловский', 'Савёловский', regex=True, inplace=True)
Moscow_housing_price_df["Borough_Name"].replace('Измайлово Северное', 'Северное Измайлово', regex=True, inplace=True)
Moscow_housing_price_df["Borough_Name"].replace('Медведково Северное', 'Северное Медведково', regex=True, inplace=True)
Moscow_housing_price_df["Borough_Name"].replace('Тушино Северное', 'Северное Тушино', regex=True, inplace=True)
Moscow_housing_price_df["Borough_Name"].replace('Теплый Стан', 'Тёплый Стан', regex=True, inplace=True)
Moscow_housing_price_df["Borough_Name"].replace('Тропарево-Никулино', 'Тропарёво-Никулино', regex=True, inplace=True)
Moscow_housing_price_df["Borough_Name"].replace('Филевский Парк', 'Филёвский Парк', regex=True, inplace=True)
Moscow_housing_price_df["Borough_Name"].replace('Хорошево-Мневники', 'Хорошёво-Мнёвники', regex=True, inplace=True)
Moscow_housing_price_df["Borough_Name"].replace('Хорошевский', 'Хорошёвский', regex=True, inplace=True)
Moscow_housing_price_df["Borough_Name"].replace('Черемушки', 'Черёмушки', regex=True, inplace=True)
Moscow_housing_price_df["Borough_Name"].replace('Медведково Южное', 'Южное Медведково', regex=True, inplace=True)
Moscow_housing_price_df["Borough_Name"].replace('Тушино Южное', 'Южное Тушино', regex=True, inplace=True)
Moscow_housing_price_df["Borough_Name"].replace('Мосрентген', '"Мосрентген"', regex=True, inplace=True)
Moscow_housing_price_df["Borough_Name"].replace('Бутово Северное', 'Северное Бутово', regex=True, inplace=True)
Moscow_housing_price_df["Borough_Name"].replace('Бутово Южное', 'Южное Бутово', regex=True, inplace=True)
Moscow_housing_price_df["Borough_Name"].replace('Десеновское', 'Десёновское', regex=True, inplace=True)
Moscow_housing_price_df["Borough_Name"].replace('Кленовское', 'Клёновское', regex=True, inplace=True)
Moscow_housing_price_df["Borough_Name"].replace('Новофедоровское', 'Новофёдоровское', regex=True, inplace=True)

Clear the dataframe


#### Here we download data with Moscow Boroughs Population Density

In [14]:

url = "https://ru.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D1%80%D0%B0%D0%B9%D0%BE%D0%BD%D0%BE%D0%B2_%D0%B8_%D0%BF%D0%BE%D1%81%D0%B5%D0%BB%D0%B5%D0%BD%D0%B8%D0%B9_%D0%9C%D0%BE%D1%81%D0%BA%D0%B2%D1%8B"

try:
    print('Try to request url "{}"'.format(url))
    response = requests.get(url)
    print('Url "{}" requested. '.format(url))

    # Take first HTML table with districts from the page and parse it into dataframe
    print('Try to extract dataset from HTML table'.format(url))
    soup = BeautifulSoup(response.text, 'html.parser')
    tables = soup.findAll('table', { 'class' : 'standard sortable' }, limit=1) 
    Moscow_dens_df = parse_html_table(tables[0]) 
    print('Success extract dataset from HTML table'.format(url))

    # Save dataframe for future use
    Moscow_dens_df.to_csv("data\Moscow_dens_df_raw.csv", index = False)

except Exception as err: 
    print('Request Url "{}" failed. Load previously saved dataframe'.format(url))
    print('Error is: {}'.format(err))

Try to request url "https://ru.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D1%80%D0%B0%D0%B9%D0%BE%D0%BD%D0%BE%D0%B2_%D0%B8_%D0%BF%D0%BE%D1%81%D0%B5%D0%BB%D0%B5%D0%BD%D0%B8%D0%B9_%D0%9C%D0%BE%D1%81%D0%BA%D0%B2%D1%8B"
Url "https://ru.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D1%80%D0%B0%D0%B9%D0%BE%D0%BD%D0%BE%D0%B2_%D0%B8_%D0%BF%D0%BE%D1%81%D0%B5%D0%BB%D0%B5%D0%BD%D0%B8%D0%B9_%D0%9C%D0%BE%D1%81%D0%BA%D0%B2%D1%8B" requested. 
Try to extract dataset from HTML table
Success extract dataset from HTML table


In [15]:
Moscow_dens_df

,№,Флаг,Герб,Название района[2]/поселения[3][4],Название cоответствующего внутригородского муниципального образования: муниципального округа / поселения / городского округа[5],Адми-нистра-тивныйокруг,"Пло-щадь,[6][7]км²","Насе-ление2020[8],чел.","Плот-ностьнасе-ления2020,чел. / км²","Пло-щадьжилого фонда(01.01.2010)[9],тыс. м²","Жил-площадьначело-века(01.01.2010),м²/чел."
0,1.0,,,Академический,Академический,ЮЗАО,"5,83",↗110 038,18874.44,"2467,0","22,7\n"
1,2.0,,,Алексеевский,Алексеевский,СВАО,"5,29",↗80 634,15242.72,"1607,9","20,5\n"
2,3.0,,,Алтуфьевский,Алтуфьевский,СВАО,"3,25",↗57 697,17752.92,"839,3","15,5\n"
3,4.0,,,Арбат,Арбат,ЦАО,"2,11",↗36 308,17207.58,"731,0","26,0\n"
4,5.0,,,Аэропорт,Аэропорт,САО,"4,58",↗79 541,17367.03,"1939,7","25,9\n"
...,...,...,...,...,...,...,...,...,...,...,...
141,142.0,,,"Сосенское, поселение[Прим. 17]","Сосенское, поселение",НАО,"67,07",↗33 096,493.45,,\n
142,143.0,,,"Троицк, поселение[Прим. 18]","Троицк, городской округ",ТАО,"16,33",↗61 284,3752.85,,\n
143,144.0,,,"Филимонковское, поселение[Прим. 19]","Филимонковское, поселение",НАО,"35,77",↗8017,224.13,,\n
144,145.0,,,"Щаповское, поселение[Прим. 20]","Щаповское, поселение",ТАО,"86,06",↗9601,111.56,,\n


#### Not bad, but strongly needed to cleaning this dataframe

In [16]:
print('Clear the dataframe')

# Drop some unused colums 
Moscow_dens_df.drop([Moscow_dens_df.columns[0], Moscow_dens_df.columns[1], Moscow_dens_df.columns[2], Moscow_dens_df.columns[3], Moscow_dens_df.columns[5]], axis=1, inplace=True)

# Set columns for dataframe
Moscow_dens_df.columns=["Borough_Name", "Borough_Area", "Borough_Population", "Borough_Population_Density", "Borough_Housing_Area", "Borough_Housing_Area_Per_Person"]

# Clear Borough Name from additional information
Moscow_dens_df["Borough_Name"].replace(', поселение ', '', regex=True, inplace=True)
Moscow_dens_df["Borough_Name"].replace(', городской округ ', '', regex=True, inplace=True)
Moscow_dens_df["Borough_Name"] = Moscow_dens_df["Borough_Name"].str.strip(' \n\t')
Moscow_dens_df["Borough_Name"].replace('Мосрентген', '"Мосрентген"', regex=True, inplace=True)

# Replace '\n' and ' ↗' in some columns
Moscow_dens_df.replace('\n', '', regex=True, inplace=True)
Moscow_dens_df.replace('↗', '', regex=True, inplace=True)
Moscow_dens_df.replace('↘', '', regex=True, inplace=True)

# Delete extra spaces in numeric columns
Moscow_dens_df["Borough_Area"].replace(' ', '', regex=True, inplace=True)
Moscow_dens_df["Borough_Population"].replace('\xa0', '', regex=True, inplace=True)
Moscow_dens_df["Borough_Population"].replace(' ', '', regex=True, inplace=True)
Moscow_dens_df["Borough_Population_Density"].replace(' ', '', regex=True, inplace=True)
Moscow_dens_df["Borough_Housing_Area"].replace(' ', '', regex=True, inplace=True)
Moscow_dens_df["Borough_Housing_Area_Per_Person"].replace(' ', '', regex=True, inplace=True)

# Replace ',' to '.' for float columns
Moscow_dens_df["Borough_Area"].replace(',', '.', regex=True, inplace=True)
Moscow_dens_df["Borough_Housing_Area"].replace(',', '.', regex=True, inplace=True)
Moscow_dens_df["Borough_Housing_Area_Per_Person"].replace(',', '.', regex=True, inplace=True)

# Convert from string to numeric
Moscow_dens_df["Borough_Population"] = Moscow_dens_df["Borough_Population"].astype(int)
Moscow_dens_df["Borough_Population_Density"] = Moscow_dens_df["Borough_Population_Density"].astype(int)
Moscow_dens_df["Borough_Area"] = Moscow_dens_df["Borough_Area"].astype(float)
Moscow_dens_df['Borough_Housing_Area'] = pd.to_numeric(Moscow_dens_df['Borough_Housing_Area'], errors='coerce')
Moscow_dens_df['Borough_Housing_Area_Per_Person'] = pd.to_numeric(Moscow_dens_df['Borough_Housing_Area_Per_Person'], errors='coerce')

# Take a look at the dataframe
print('Take a look at the dataframe')
print(Moscow_dens_df.head())
print(Moscow_dens_df.shape)

print('Take a look at the dataframe data types')
print(Moscow_dens_df.dtypes)

# Save copy of the dataframe
Moscow_dens_df.to_csv("data\Moscow_dens_df.csv", index = False)

Clear the dataframe
Take a look at the dataframe
    Borough_Name  Borough_Area  Borough_Population  \
0  Академический          5.83              110038   
1   Алексеевский          5.29               80634   
2   Алтуфьевский          3.25               57697   
3          Арбат          2.11               36308   
4       Аэропорт          4.58               79541   

   Borough_Population_Density  Borough_Housing_Area  \
0                       18874                2467.0   
1                       15242                1607.9   
2                       17752                 839.3   
3                       17207                 731.0   
4                       17367                1939.7   

   Borough_Housing_Area_Per_Person  
0                             22.7  
1                             20.5  
2                             15.5  
3                             26.0  
4                             25.9  
(146, 6)
Take a look at the dataframe data types
Borough_Name            

#### Here we are merge data from all sources gathered to one dataframe

In [17]:
# Merge datasets
Moscow_Borough_df = pd.merge(left=Moscow_df, right=Moscow_dens_df, how='left', left_on='Borough_Name', right_on='Borough_Name')
Moscow_Borough_df = pd.merge(left=Moscow_Borough_df, right=Moscow_coord_df, how='left', left_on='Borough_Name', right_on='Borough_Name')
Moscow_Borough_df = pd.merge(left=Moscow_Borough_df, right=Moscow_housing_price_df, how='left', left_on='Borough_Name', right_on='Borough_Name')

# We do not have statistics on “housing prices” and “housing area” for all boroughs, so we exclude these boroughs from our analysis
print('Print Boroughs without Housing Price')
Moscow_Borough_df[pd.isnull(Moscow_Borough_df['Borough_Housing_Price'])]
print('Delete Boroughs without Housing Price')
Moscow_Borough_df.dropna(subset=['Borough_Housing_Price'], inplace=True)

print('Print Boroughs without Housing Area')
Moscow_Borough_df[pd.isnull(Moscow_Borough_df['Borough_Housing_Area'])]
print('Delete Boroughs without Housing Area')
Moscow_Borough_df.dropna(subset=['Borough_Housing_Area'], inplace=True)

# reset index
Moscow_Borough_df.reset_index(drop=True, inplace=True)

Print Boroughs without Housing Price
Delete Boroughs without Housing Price
Print Boroughs without Housing Area
Delete Boroughs without Housing Area


In [18]:
Moscow_Bor_df = Moscow_Borough_df.dropna()
print("Detail_info_about_borough")
Moscow_Bor_df

Detail_info_about_borough


,Borough_index,Borough_Name,District_Name,Borough_Type,OKATO_Borough_Code,OKTMO_District_Code,Borough_Area,Borough_Population,Borough_Population_Density,Borough_Housing_Area,Borough_Housing_Area_Per_Person,Latitude,Longitude,Borough_Housing_Price
0,1.0,Академический,ЮЗАО,Муниципальный округ,45293554,45397000,5.83,110038,18874,2467.0,22.7,55.678645,37.571582,199999.0
1,2.0,Алексеевский,СВАО,Муниципальный округ,45280552,45349000,5.29,80634,15242,1607.9,20.5,55.825026,37.655436,199474.0
2,3.0,Алтуфьевский,СВАО,Муниципальный округ,45280554,45350000,3.25,57697,17752,839.3,15.5,55.874212,37.576297,138021.0
4,5.0,Аэропорт,САО,Муниципальный округ,45277553,45333000,4.58,79541,17367,1939.7,25.9,55.800402,37.533156,234544.0
5,6.0,Бабушкинский,СВАО,Муниципальный округ,45280556,45351000,5.07,88691,17493,1586.3,18.5,55.880056,37.667719,164324.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,121.0,Южное Тушино,СЗАО,Муниципальный округ,45283593,45373000,7.94,109708,13817,1714.9,16.9,55.839948,37.424313,162854.0
116,122.0,Южнопортовый,ЮВАО,Муниципальный округ,45290594,45396000,4.53,74841,16521,1449.0,20.6,55.728970,37.662553,190357.0
117,123.0,Якиманка,ЦАО,Муниципальный округ,45286596,45384000,4.80,27911,5814,788.7,29.8,55.740329,37.620661,404471.0
118,124.0,Ярославский,СВАО,Муниципальный округ,45280597,45365000,7.99,98272,12299,1404.5,15.3,55.879174,37.712051,159241.0


### Now we finally visualizing our data of Moscow borough

In [27]:
map_moscow = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for Borough_Name, lat, lng, Borough_Population in zip(Moscow_Bor_df['Borough_Name'], 
                                                      Moscow_Bor_df['Latitude'], 
                                                      Moscow_Bor_df['Longitude'], 
                                                      Moscow_Bor_df['Borough_Population']):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5, # define how big you want the circle markers to be
        color='green',
        fill=True,
        #popup='{}, Москва, Россия ({:})'.format(Borough_Name, Borough_Population),
        popup=folium.Popup('{},Численность населения - {:}'.format(Borough_Name, Borough_Population), parse_html=True),
        fill_color='green',
        fill_opacity=0.6
    ).add_to(map_moscow)

# display map
map_moscow

## Connect Foursquire API

#### In this module we connect outside API (Foursquare), to provide data

In [ ]:
CLIENT_ID = 'DAZHEWX2X4ACOILUYLMHII0CDVZ5M0CBGLSUDV3PT4IXBYAO' # your Foursquare ID
CLIENT_SECRET = 'PEABAZHM35ODZK1UNGJMCYD02VFBUWFHSKX2CWSHXFC1CA2N' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [29]:
neighborhood_latitude = Moscow_Bor_df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Moscow_Bor_df.loc[0, 'Longitude'] # neighborhood longitude value
neighborhood_name = Moscow_Bor_df.loc[0, 'Borough_Name'] # neighborhood name
print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Академический are 55.67864455, 37.57158238499227.


#### Requesting data from foursquare

In [30]:
LIMIT = 50
radius = 300
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    VERSION, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=DAZHEWX2X4ACOILUYLMHII0CDVZ5M0CBGLSUDV3PT4IXBYAO&client_secret=PEABAZHM35ODZK1UNGJMCYD02VFBUWFHSKX2CWSHXFC1CA2N&ll=55.67864455,37.57158238499227&v=20180605&radius=300&limit=50'

In [31]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60720920bb08722761c01e94'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Akademichesky',
  'headerFullLocation': 'Akademichesky, Moscow',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 12,
  'suggestedBounds': {'ne': {'lat': 55.681344552700004,
    'lng': 37.57636209300854},
   'sw': {'lat': 55.6759445473, 'lng': 37.566802676975996}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '590f14b4a4b51b09214a13f0',
       'name': 'HookahPlace Profsoyuz / Хукаплейс Профсоюз (ХукаПлейс Профсоюз)',
       'location': {'address': 'ул. Кржижановского, 15, корп. 2',
        'lat': 55.68031744579531,
        'lng': 37.57188592111364,
    

In [32]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [33]:

# Clean the json and structure it into a dataframe.
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,name,categories,lat,lng
0,HookahPlace Profsoyuz / Хукаплейс Профсоюз (Ху...,Hookah Bar,55.680317,37.571886
1,Эндорфин квест,Escape Room,55.680578,37.570033
2,PowerLine,Electronics Store,55.677450,37.567905
3,Стара Юга,Eastern European Restaurant,55.679777,37.570949
4,ВьетКафе,Vietnamese Restaurant,55.677810,37.574537
5,Кулинарная Лавка Братьев Караваевых,Café,55.677802,37.574302
6,Ароматный мир,Wine Shop,55.677771,37.573982
7,Батони,Caucasian Restaurant,55.677772,37.574114
8,Бильярд Шаровня,Pool Hall,55.676396,37.570070
9,Кинто,Caucasian Restaurant,55.680280,37.570280


#### Founding data look pretty accuared. Now continue to provide data for all borough.

In [34]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

12 venues were returned by Foursquare.


In [35]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough_Name', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [37]:
moscow_venues = getNearbyVenues(names=Moscow_Bor_df['Borough_Name'],
                                latitudes=Moscow_Bor_df["Latitude"],
                                longitudes=Moscow_Bor_df['Longitude'])

Академический
Алексеевский
Алтуфьевский
Аэропорт
Бабушкинский
Басманный
Беговой
Бескудниковский
Бибирево
Богородское
Внуково
Войковский
Восточное Дегунино
Восточный
Головинский
Даниловский
Дмитровский
Донской
Дорогомилово
Замоскворечье
Западное Дегунино
Зюзино
Зябликово
Ивановское
Измайлово
Коньково
Коптево
Красносельский
Крылатское
Кузьминки
Кунцево
Левобережный
Лефортово
Ломоносовский
Лосиноостровский
Люблино
Марьина Роща
Марьино
Метрогородок
Митино
Можайский
Молжаниновский
Москворечье-Сабурово
Нагатино-Садовники
Новогиреево
Новокосино
Обручевский
Орехово-Борисово Северное
Орехово-Борисово Южное
Останкинский
Отрадное
Очаково-Матвеевское
Печатники
Покровское-Стрешнево
Пресненский
Проспект Вернадского
Раменки
Ростокино
Рязанский
Савёловский
Свиблово
Северное Медведково
Северное Тушино
Северный
Сокол
Соколиная Гора
Сокольники
Солнцево
Строгино
Таганский
Текстильщики
Тёплый Стан
Тимирязевский
Филёвский Парк
Фили-Давыдково
Хамовники
Ховрино
Хорошёво-Мнёвники
Хорошёвский
Черёмушки
Чертанов

In [38]:
print(moscow_venues.shape)
moscow_venues

(1549, 7)


,Borough_Name,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Академический,55.678645,37.571582,HookahPlace Profsoyuz / Хукаплейс Профсоюз (Ху...,55.680317,37.571886,Hookah Bar
1,Академический,55.678645,37.571582,Эндорфин квест,55.680578,37.570033,Escape Room
2,Академический,55.678645,37.571582,Natura Siberica,55.677181,37.565778,Cosmetics Shop
3,Академический,55.678645,37.571582,Il Pittore,55.674708,37.570920,Italian Restaurant
4,Академический,55.678645,37.571582,PowerLine,55.677450,37.567905,Electronics Store
...,...,...,...,...,...,...,...
1544,Ясенево,55.593787,37.532633,Карамзина 1,55.593920,37.538547,Bus Stop
1545,Ясенево,55.593787,37.532633,"остановка ""Ясенево"" на МКАДе",55.590460,37.534158,Bus Stop
1546,Ясенево,55.593787,37.532633,Виал Трейд,55.591010,37.537266,Convenience Store
1547,Ясенево,55.593787,37.532633,Площадка На Голубинской 25,55.597952,37.534351,Playground


In [39]:
moscow_venues.groupby('Borough_Name').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Borough_Name,,,,,,
Академический,24,24,24,24,24,24
Алексеевский,23,23,23,23,23,23
Алтуфьевский,4,4,4,4,4,4
Аэропорт,50,50,50,50,50,50
Бабушкинский,15,15,15,15,15,15
...,...,...,...,...,...,...
Южное Тушино,6,6,6,6,6,6
Южнопортовый,19,19,19,19,19,19
Якиманка,50,50,50,50,50,50


In [40]:
print('There are {} uniques categories.'.format(len(moscow_venues['Venue Category'].unique())))

There are 242 uniques categories.


#### For using machine learning algorithm, like clustering, fisrt need do preprocess encoding

In [41]:
moscow_onehot = pd.get_dummies(moscow_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
moscow_onehot['Borough_Name'] = moscow_venues['Borough_Name'] 

# move neighborhood column to the first column
fixed_columns = [moscow_onehot.columns[-1]] + list(moscow_onehot.columns[:-1])
moscow_onehot = moscow_onehot[fixed_columns]

moscow_onehot.head()

,Borough_Name,Accessories Store,American Restaurant,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,...,Vietnamese Restaurant,Warehouse Store,Watch Shop,Waterfall,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Академический,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Академический,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Академический,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Академический,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Академический,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
moscow_onehot.shape

(1549, 243)

In [43]:
moscow_grouped = moscow_onehot.groupby('Borough_Name').mean().reset_index()
moscow_grouped

,Borough_Name,Accessories Store,American Restaurant,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,...,Vietnamese Restaurant,Warehouse Store,Watch Shop,Waterfall,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Академический,0.0,0.0,0.000000,0.0,0.00,0.00,0.0,0.0,0.0,...,0.041667,0.0,0.0,0.0,0.0,0.0,0.041667,0.0,0.0,0.000000
1,Алексеевский,0.0,0.0,0.043478,0.0,0.00,0.00,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.043478
2,Алтуфьевский,0.0,0.0,0.000000,0.0,0.00,0.00,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
3,Аэропорт,0.0,0.0,0.000000,0.0,0.00,0.00,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
4,Бабушкинский,0.0,0.0,0.066667,0.0,0.00,0.00,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,Южное Тушино,0.0,0.0,0.000000,0.0,0.00,0.00,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
82,Южнопортовый,0.0,0.0,0.000000,0.0,0.00,0.00,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.052632
83,Якиманка,0.0,0.0,0.000000,0.0,0.02,0.02,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.020000
84,Ярославский,0.0,0.0,0.000000,0.0,0.00,0.00,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000


#### Very good, encoding finish well, data prepared to using ML-techniques.

In [44]:
moscow_grouped.shape

(86, 243)

In [45]:
num_top_venues = 5

for hood in moscow_grouped['Borough_Name']:
    print("----"+hood+"----")
    temp = moscow_grouped[moscow_grouped['Borough_Name'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Академический----
                venue  freq
0          Shoe Store  0.08
1  Italian Restaurant  0.08
2      Cosmetics Shop  0.08
3    Sushi Restaurant  0.04
4           Pet Store  0.04


----Алексеевский----
                  venue  freq
0             Bookstore  0.09
1             Pet Store  0.09
2         Jewelry Store  0.04
3                Bridge  0.04
4  Caucasian Restaurant  0.04


----Алтуфьевский----
                   venue  freq
0          Auto Workshop  0.50
1          Grocery Store  0.25
2            Supermarket  0.25
3      Accessories Store  0.00
4  Performing Arts Venue  0.00


----Аэропорт----
               venue  freq
0        Coffee Shop  0.14
1     Cosmetics Shop  0.10
2               Café  0.08
3        Flower Shop  0.04
4  Health Food Store  0.02


----Бабушкинский----
           venue  freq
0           Park  0.13
1     Skate Park  0.13
2     Playground  0.07
3  Go Kart Track  0.07
4   Tennis Court  0.07


----Басманный----
                  venue  freq
0  Gym

In [46]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [47]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough_Name']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Borough_Name'] = moscow_grouped['Borough_Name']

for ind in np.arange(moscow_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(moscow_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Borough_Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Академический,Shoe Store,Italian Restaurant,Cosmetics Shop,Sushi Restaurant,Pet Store,Pharmacy,Movie Theater,Supermarket,Hookah Bar,Café
1,Алексеевский,Bookstore,Pet Store,Jewelry Store,Bridge,Caucasian Restaurant,Clothing Store,Convenience Store,Historic Site,Hookah Bar,Japanese Restaurant
2,Алтуфьевский,Auto Workshop,Grocery Store,Supermarket,Accessories Store,Performing Arts Venue,Opera House,Optical Shop,Other Repair Shop,Outdoor Gym,Outdoor Sculpture
3,Аэропорт,Coffee Shop,Cosmetics Shop,Café,Flower Shop,Health Food Store,Bookstore,Dessert Shop,Pie Shop,Deli / Bodega,Supermarket
4,Бабушкинский,Park,Skate Park,Playground,Go Kart Track,Tennis Court,Outdoor Gym,Gym / Fitness Center,Athletics & Sports,Soccer Field,Pool
...,...,...,...,...,...,...,...,...,...,...,...
81,Южное Тушино,Park,Bath House,Supermarket,Gastropub,Gym,Office,Optical Shop,Other Repair Shop,Outdoor Gym,Outdoor Sculpture
82,Южнопортовый,Pharmacy,Yoga Studio,Miscellaneous Shop,Coffee Shop,Gourmet Shop,Spa,Butcher,Steakhouse,Burger Joint,Health Food Store
83,Якиманка,Coffee Shop,Café,Restaurant,Bookstore,Cosmetics Shop,Dance Studio,Gym / Fitness Center,Road,Yoga Studio,Flower Shop
84,Ярославский,Eastern European Restaurant,Gym,Convenience Store,Supermarket,Soccer Field,Bookstore,Pelmeni House,Shopping Mall,Big Box Store,Train Station


In [59]:
neighborhoods_venues_sorted.iloc[43]

Borough_Name                       Новогиреево
1st Most Common Venue              Supermarket
2nd Most Common Venue                 Bus Stop
3rd Most Common Venue            Fishing Store
4th Most Common Venue                 Pharmacy
5th Most Common Venue         Arepa Restaurant
6th Most Common Venue        Health Food Store
7th Most Common Venue                     Café
8th Most Common Venue         Department Store
9th Most Common Venue     Gym / Fitness Center
10th Most Common Venue             Pizza Place
Name: 43, dtype: object

### Finally, divided our borough, depending of they type, on different clusters

In [114]:
kclusters = 4

moscow_grouped_clustering = moscow_grouped.drop('Borough_Name', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(moscow_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
/srv/conda/envs/notebook/lib/python3.7/site-

array([0, 2, 3, 0, 2, 2, 0, 0, 2, 1], dtype=int32)

In [123]:
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
moscow_data = Moscow_Bor_df[['Borough_Name','Borough_Population', 'Latitude', 'Longitude']]
moscow_merged = moscow_data

# merge downtown_grouped with manhattan_data to add latitude/longitude for each neighborhood
moscow_merged = moscow_merged.join(neighborhoods_venues_sorted.set_index('Borough_Name'), on='Borough_Name')

moscow_merged =moscow_merged.dropna()
moscow_merged

,Borough_Name,Borough_Population,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Академический,110038,55.678645,37.571582,0.0,Shoe Store,Italian Restaurant,Cosmetics Shop,Sushi Restaurant,Pet Store,Pharmacy,Movie Theater,Supermarket,Hookah Bar,Café
1,Алексеевский,80634,55.825026,37.655436,2.0,Bookstore,Pet Store,Jewelry Store,Bridge,Caucasian Restaurant,Clothing Store,Convenience Store,Historic Site,Hookah Bar,Japanese Restaurant
2,Алтуфьевский,57697,55.874212,37.576297,3.0,Auto Workshop,Grocery Store,Supermarket,Accessories Store,Performing Arts Venue,Opera House,Optical Shop,Other Repair Shop,Outdoor Gym,Outdoor Sculpture
4,Аэропорт,79541,55.800402,37.533156,0.0,Coffee Shop,Cosmetics Shop,Café,Flower Shop,Health Food Store,Bookstore,Dessert Shop,Pie Shop,Deli / Bodega,Supermarket
5,Бабушкинский,88691,55.880056,37.667719,2.0,Park,Skate Park,Playground,Go Kart Track,Tennis Court,Outdoor Gym,Gym / Fitness Center,Athletics & Sports,Soccer Field,Pool
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,Южное Тушино,109708,55.839948,37.424313,2.0,Park,Bath House,Supermarket,Gastropub,Gym,Office,Optical Shop,Other Repair Shop,Outdoor Gym,Outdoor Sculpture
116,Южнопортовый,74841,55.728970,37.662553,0.0,Pharmacy,Yoga Studio,Miscellaneous Shop,Coffee Shop,Gourmet Shop,Spa,Butcher,Steakhouse,Burger Joint,Health Food Store
117,Якиманка,27911,55.740329,37.620661,0.0,Coffee Shop,Café,Restaurant,Bookstore,Cosmetics Shop,Dance Studio,Gym / Fitness Center,Road,Yoga Studio,Flower Shop
118,Ярославский,98272,55.879174,37.712051,2.0,Eastern European Restaurant,Gym,Convenience Store,Supermarket,Soccer Field,Bookstore,Pelmeni House,Shopping Mall,Big Box Store,Train Station


#### Visualing our clusters

In [124]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(moscow_merged['Latitude'], 
                                  moscow_merged['Longitude'], 
                                  moscow_merged['Borough_Name'], 
                                  moscow_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Кластер ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Showing data in different clusters

In [126]:
moscow_merged.loc[moscow_merged['Cluster Labels'] == 0, moscow_merged.columns[[0] + list(range(5, moscow_merged.shape[1]))]]

,Borough_Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Академический,Shoe Store,Italian Restaurant,Cosmetics Shop,Sushi Restaurant,Pet Store,Pharmacy,Movie Theater,Supermarket,Hookah Bar,Café
4,Аэропорт,Coffee Shop,Cosmetics Shop,Café,Flower Shop,Health Food Store,Bookstore,Dessert Shop,Pie Shop,Deli / Bodega,Supermarket
7,Беговой,Cafeteria,Restaurant,Convenience Store,Other Repair Shop,Dumpling Restaurant,Camera Store,Park,Breakfast Spot,Coffee Shop,Hookah Bar
8,Бескудниковский,Japanese Restaurant,Liquor Store,Bike Shop,Farmers Market,Pizza Place,Music Store,Convenience Store,Eastern European Restaurant,Beer Bar,Park
17,Войковский,Cosmetics Shop,Coffee Shop,Yoga Studio,Boutique,Board Shop,Ice Cream Shop,Health Food Store,Fast Food Restaurant,Steakhouse,Café
18,Восточное Дегунино,Ice Cream Shop,Bus Stop,Bank,Gym,Café,Moving Target,Pizza Place,Paper / Office Supplies Store,Pelmeni House,Pedestrian Plaza
20,Восточный,Japanese Restaurant,Hotel,Hotel Bar,Clothing Store,Middle Eastern Restaurant,Concert Hall,Theme Park,Theme Park Ride / Attraction,Blini House,Shawarma Place
25,Даниловский,Coffee Shop,Dance Studio,Cosmetics Shop,Bakery,Deli / Bodega,Buffet,Indian Restaurant,Gelato Shop,Brewery,Nightclub
27,Донской,Coffee Shop,Gym / Fitness Center,Lounge,Public Art,Buffet,Soccer Stadium,Pool,Nightclub,Gym,Hotel
28,Дорогомилово,Coffee Shop,Café,Flower Shop,Ice Cream Shop,Gym / Fitness Center,Bus Stop,Climbing Gym,Other Repair Shop,Pie Shop,Health & Beauty Service


In [127]:
moscow_merged.loc[moscow_merged['Cluster Labels'] == 1, moscow_merged.columns[[0] + list(range(5, moscow_merged.shape[1]))]]

,Borough_Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Богородское,Pharmacy,Supermarket,Tram Station,Gym / Fitness Center,Health Food Store,Ice Cream Shop,Food & Drink Shop,Convenience Store,Bus Stop,Paintball Field
16,Внуково,Train Station,Construction & Landscaping,Bar,Bus Stop,Farm,Performing Arts Venue,Pelmeni House,Pedestrian Plaza,Park,Accessories Store
32,Зябликово,Supermarket,Bus Stop,Soccer Field,BBQ Joint,Accessories Store,Performing Arts Venue,Optical Shop,Other Repair Shop,Outdoor Gym,Outdoor Sculpture
37,Коптево,Bus Stop,Supermarket,Auto Workshop,Historic Site,Convenience Store,Pharmacy,Café,Park,Caucasian Restaurant,Pelmeni House
49,Лосиноостровский,Bus Stop,Restaurant,Supermarket,Bath House,Public Art,Convenience Store,Pharmacy,Park,Grocery Store,Gym / Fitness Center
50,Люблино,Stadium,Bus Line,Bus Stop,Shawarma Place,Pharmacy,Food & Drink Shop,Performing Arts Venue,Other Repair Shop,Outdoor Gym,Outdoor Sculpture
54,Метрогородок,Bus Stop,Grocery Store,Photography Studio,Sushi Restaurant,Tram Station,Wine Shop,Optical Shop,Other Repair Shop,Opera House,Pelmeni House
69,Орехово-Борисово Северное,Supermarket,Bus Stop,Soccer Field,BBQ Joint,Accessories Store,Performing Arts Venue,Optical Shop,Other Repair Shop,Outdoor Gym,Outdoor Sculpture
70,Орехово-Борисово Южное,Bus Stop,Pharmacy,Bookstore,Middle Eastern Restaurant,Martial Arts School,Gym / Fitness Center,Food & Drink Shop,Convenience Store,Supermarket,Eastern European Restaurant
84,Свиблово,Sporting Goods Shop,Dance Studio,Bus Stop,Convenience Store,Park,Accessories Store,Pet Store,Outdoor Gym,Outdoor Sculpture,Paintball Field


In [132]:
moscow_merged.loc[moscow_merged['Cluster Labels'] == 2, moscow_merged.columns[[0] + list(range(5, moscow_merged.shape[1]))]]

,Borough_Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Алексеевский,Bookstore,Pet Store,Jewelry Store,Bridge,Caucasian Restaurant,Clothing Store,Convenience Store,Historic Site,Hookah Bar,Japanese Restaurant
5,Бабушкинский,Park,Skate Park,Playground,Go Kart Track,Tennis Court,Outdoor Gym,Gym / Fitness Center,Athletics & Sports,Soccer Field,Pool
6,Басманный,Gym / Fitness Center,History Museum,Hookah Bar,Motorcycle Shop,Gift Shop,Gun Range,Caucasian Restaurant,Pub,Park,Supermarket
9,Бибирево,Pilates Studio,Shawarma Place,Food & Drink Shop,Park,Auto Workshop,Historic Site,Accessories Store,Pelmeni House,Pedestrian Plaza,Paper / Office Supplies Store
23,Головинский,Waterfall,Middle Eastern Restaurant,Bus Line,Lake,Park,Palace,Performing Arts Venue,Pelmeni House,Pedestrian Plaza,Paper / Office Supplies Store
26,Дмитровский,Supermarket,Auto Workshop,Restaurant,Pizza Place,Café,Gym,Park,Modern European Restaurant,Bath House,Convenience Store
33,Ивановское,Pizza Place,Supermarket,Liquor Store,Skating Rink,Food & Drink Shop,Park,Gym Pool,Café,Track,Electronics Store
34,Измайлово,Boat Rental,Fountain,Theme Park Ride / Attraction,Music Venue,Lake,Plaza,Playground,Ice Cream Shop,Pedestrian Plaza,Snack Place
43,Кунцево,Bus Line,Cosmetics Shop,Theater,Juice Bar,Gym / Fitness Center,Bus Stop,Gym,Supermarket,Gym Pool,Spa
48,Ломоносовский,Café,Playground,Food & Drink Shop,Grocery Store,Trail,Wine Shop,Church,Italian Restaurant,Gym / Fitness Center,Theater


In [131]:
moscow_merged.loc[moscow_merged['Cluster Labels'] == 3, moscow_merged.columns[[0] + list(range(5, moscow_merged.shape[1]))]]

,Borough_Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Алтуфьевский,Auto Workshop,Grocery Store,Supermarket,Accessories Store,Performing Arts Venue,Opera House,Optical Shop,Other Repair Shop,Outdoor Gym,Outdoor Sculpture
104,Ховрино,Supermarket,Café,Auto Workshop,Park,Accessories Store,Performing Arts Venue,Opera House,Optical Shop,Other Repair Shop,Outdoor Gym
106,Хорошёвский,Auto Workshop,Martial Arts School,Hookah Bar,Music Store,Café,Park,Accessories Store,Performing Arts Venue,Pelmeni House,Pedestrian Plaza


#### Here this work is over, we are clustered Moscow on different cluster in depend of borough type 

#### Thank all how intersted my work, try to make this one interest